In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import re
import datetime
import glob
import pickle

In [12]:
paths=[r"E:\Dataset\terrain classification.v27i.folder/*",r"E:\Dataset\terrain classification1.v11i.folder",
       r"E:\Dataset\terrain classification2.v6i.folder",r"E:\Dataset\terrain classification3.v6i.folder",r"E:\Dataset\terrain classification4.v12i.folder"]
l=[]
for path in paths:
    for i in sorted(list(glob.glob(path+'/*'))):
        for j in sorted(list(glob.glob(i+'/*'))):
            if j.endswith('.jpg') or j.endswith('.JPG') or j.endswith('.png') or j.endswith('.PNG') or j.endswith('.jpeg') or j.endswith('.JPEG') or j.endswith('.tif') or j.endswith('.TIF'):
                l.append(j.split('\\')[-1])

with open('path.pkl', 'wb') as f:
    pickle.dump(l, f)

In [13]:
l

['15_jpg.rf.12a60aa473944da08ddae961e7d6ee8d.jpg',
 '18_jpg.rf.410567a589332255c17855ed99384ca4.jpg',
 '20_jpg.rf.8f47d3d61f00c016d8b428f5a3bf4e7b.jpg',
 '22_jpg.rf.e0ee003d759143176dcfcffb9459189d.jpg',
 '52_jpg.rf.73da66e7d5718879c0583f0aa6c5e92b.jpg',
 '5_jpg.rf.a912f45677b54b044b82a914874980e4.jpg',
 '65_jpg.rf.34219902b2927639095a0b7b9c12819b.jpg',
 '89_jpg.rf.07e75fa450723eb9d4292f93dd7a0fdf.jpg',
 '95_jpg.rf.3b8c2436820f243fb1a27369a1f67886.jpg',
 '1058_jpg.rf.5e7f7930c89e3da1d5122a41a1c3b2f8.jpg',
 '1059_jpg.rf.cfec15d61dbe899f3429859fc48ebf61.jpg',
 '1061_jpg.rf.dccfc10f98ad9ee13a0076ae1570a16d.jpg',
 '1068_jpg.rf.72cc3f85adf7e041cb2f937e1ff2efed.jpg',
 '1072_jpg.rf.19dee355fe6ee196ff0e7266fff896e8.jpg',
 '1077_jpg.rf.785442103cc55356047e347a9c92d9d9.jpg',
 '1080_jpg.rf.784e831552a714e63f6b4a2ff15f6c64.jpg',
 '1081_jpg.rf.6887827b6732231512a9b7afd3b45020.jpg',
 '1093_jpg.rf.1083a2106efa747c85ae1a3ac5fea984.jpg',
 '1094_jpg.rf.3f2870551e1d794aeb9fa7bdc57e820d.jpg',
 '1100_jpg.r

In [3]:
l

['gtos-mobile\\gtos-mobile\\train\\Painting',
 'gtos-mobile\\gtos-mobile\\train\\aluminum',
 'gtos-mobile\\gtos-mobile\\train\\asphalt',
 'gtos-mobile\\gtos-mobile\\train\\brick',
 'gtos-mobile\\gtos-mobile\\train\\cement',
 'gtos-mobile\\gtos-mobile\\train\\cloth',
 'gtos-mobile\\gtos-mobile\\train\\dry_leaf',
 'gtos-mobile\\gtos-mobile\\train\\glass',
 'gtos-mobile\\gtos-mobile\\train\\grass',
 'gtos-mobile\\gtos-mobile\\train\\gravel',
 'gtos-mobile\\gtos-mobile\\train\\large_limestone',
 'gtos-mobile\\gtos-mobile\\train\\leaf',
 'gtos-mobile\\gtos-mobile\\train\\metal_cover',
 'gtos-mobile\\gtos-mobile\\train\\moss',
 'gtos-mobile\\gtos-mobile\\train\\mud',
 'gtos-mobile\\gtos-mobile\\train\\paint_cover',
 'gtos-mobile\\gtos-mobile\\train\\painting_turf',
 'gtos-mobile\\gtos-mobile\\train\\paper',
 'gtos-mobile\\gtos-mobile\\train\\pebble',
 'gtos-mobile\\gtos-mobile\\train\\plastic',
 'gtos-mobile\\gtos-mobile\\train\\plastic_cover',
 'gtos-mobile\\gtos-mobile\\train\\root',
 'gto